### Contents

[Imports](#Imports)

[bottom](#bottom)

### Imports

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os
from os.path import join as pjoin
import sys 
from pathlib import Path
sys.path.insert(0, str(Path(os.getcwd()).parent))
import json 
import itertools 
from pprint import pprint
from datetime import datetime, date, timedelta
import requests 
import time
from pprint import pprint
import json 

import pandas as pd
from tqdm import tqdm
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
from fabulous import color as fb_color
from PIL import Image
from io import BytesIO
%matplotlib inline

from IPython.display import display, clear_output

In [6]:
from src.parse_utils import (
    find_card_url,
    get_images_urls_from_card_url,
    get_browse_categories_with_urls
)

In [13]:
ROOT_PATH = str(Path(os.getcwd()).parent)

'/home/aiandrejcev/wb_parsing'

In [4]:
matplotlib.rcParams['figure.figsize'] = (7, 7)
sns.set_style('whitegrid')

---

In [5]:
catalog_url = 'https://static-basket-01.wbbasket.ru/vol0/data/main-menu-ru-ru-v2.json'
catalog = requests.get(catalog_url).json()
print(len(catalog))
catalog[:2]

30


[{'id': 1234, 'name': 'Сертификаты Wildberries', 'url': '/gift/certificates'},
 {'id': 306,
  'name': 'Женщинам',
  'url': '/catalog/zhenshchinam',
  'shard': 'blackhole',
  'query': 'cat=306',
  'childs': [{'id': 8126,
    'parent': 306,
    'name': 'Блузки и рубашки',
    'seo': 'Женские блузки и рубашки',
    'url': '/catalog/zhenshchinam/odezhda/bluzki-i-rubashki',
    'shard': 'bl_shirts',
    'query': 'cat=8126'},
   {'id': 8127,
    'parent': 306,
    'name': 'Брюки',
    'seo': 'Женские брюки',
    'url': '/catalog/zhenshchinam/odezhda/bryuki-i-shorty',
    'shard': 'pants',
    'query': 'cat=8127'},
   {'id': 63010,
    'parent': 306,
    'name': 'Верхняя одежда',
    'seo': 'Верхняя женская одежда',
    'url': '/catalog/zhenshchinam/odezhda/verhnyaya-odezhda',
    'shard': 'outwear1',
    'query': 'cat=63010'},
   {'id': 8130,
    'parent': 306,
    'name': 'Джемперы, водолазки и кардиганы',
    'seo': 'Женские джемперы и кардиганы',
    'url': '/catalog/zhenshchinam/odezhda/

In [34]:
# ### dfs catalog
# from copy import deepcopy
# from collections import deque

# catalog_copy = deque([({'id': 'root', 'name': 'root', 'shard': 'root', 'query': 'root'}, subsection, 0) for subsection in deepcopy(catalog)])
# while catalog_copy:
#     parent, child, depth = catalog_copy.popleft()
#     print(f"{' '*depth}{parent['name']} shard: {parent['shard']} query: {parent['query']}")
#     if 'childs' in child:
#         catalog_copy.extendleft([
#             (
#                 {
#                     'id': child['id'], 
#                     'name': child['name'],
#                     'shard': child['shard'], 
#                     'query': child['query'],
#                 },
#                 child_,
#                 depth + 1
#             )
#             for child_ in child['childs']
#         ])

In [21]:
category = (
    catalog
    [4]['childs'][0]
)
# # pprint(category)
# browse_result_url = (
#     'https://catalog.wb.ru/catalog/' 
#     + category['shard']
#     + f'/v2/catalog?ab_testing=false&appType=1&curr=rub&dest=-446116&sort=popular&spp=30&uclusters=0'
#     + f'&{category["query"]}'
# )

from src.parse_utils import construct_browse_url

browse_result_url = construct_browse_url(
    query=category["query"],
    shard=category['shard']
)
print(browse_result_url)
browse_result_items = json.loads(requests.get(browse_result_url).content)
print(len(browse_result_items['data']['products']))
pprint(browse_result_items['data']['products'][:2])

https://catalog.wb.ru/catalog/men_clothes1/v2/catalog?ab_testing=false&appType=1&curr=rub&dest=-446116&sort=popular&spp=30&uclusters=0&cat=8144
100
[{'__sort': 81508,
  'brand': 'Style Driven',
  'brandId': 311386768,
  'colors': [{'id': 0, 'name': 'черный'}],
  'dist': 110,
  'dtype': 4,
  'entity': 'Брюки',
  'feedbacks': 304,
  'id': 250651528,
  'kindId': 1,
  'ksort': 5746,
  'logs': 'MQTo1s/4pLwzVVRh572Q9+4h8P9Nrj/ykLKfjOc5hUjkrU0Z6QMSnTkR7DcOYQF01UA',
  'meta': {},
  'name': 'Брюки классические широкие оверсайз',
  'panelPromoId': 189656,
  'pics': 12,
  'promoTextCard': 'ЕЖЕВЫГОДА',
  'promoTextCat': 'ЕЖЕВЫГОДА',
  'rating': 5,
  'reviewRating': 4.7,
  'root': 226323025,
  'siteBrandId': 0,
  'sizes': [{'dtype': 4,
             'name': '46',
             'optionId': 391936953,
             'origName': 'S',
             'payload': '0VWnUeVFf5kuVfsfVIBdRqRmrc0kJT7B0HMszip3Q5/nOWUHkdXMnDHYWitBHO08i8Wa2ixs4IiqpDWLkw',
             'price': {'basic': 836500,
                       '

In [15]:
from src.parse_utils import get_browse_categories_with_urls

flatten_categories_with_urls = get_browse_categories_with_urls(catalog=catalog)
json.dump(flatten_categories_with_urls, open(pjoin(ROOT_PATH, 'raw_data/flatten_categories_with_urls.json'), mode='w', encoding='utf-8'))
print(f"#flatten_categories_with_urls: {len(flatten_categories_with_urls):,}")
print(f"#flatten_categories_with_urls (with urls): {sum([category_info.get('url') is not None for category_info in flatten_categories_with_urls]):,}")
pprint(flatten_categories_with_urls[:15])

#flatten_categories_with_urls: 2,398
#flatten_categories_with_urls (with urls): 2,264
[{'category_id': 1234,
  'name': 'Сертификаты Wildberries',
  'query': None,
  'shard': None},
 {'category_id': 306,
  'name': 'Женщинам',
  'query': 'cat=306',
  'shard': 'blackhole'},
 {'category_id': 8126,
  'name': 'Блузки и рубашки',
  'query': 'cat=8126',
  'shard': 'bl_shirts',
  'url': 'https://catalog.wb.ru/catalog/bl_shirts/v2/catalog?ab_testing=false&appType=1&curr=rub&dest=-446116&sort=popular&spp=30&uclusters=0&cat=8126'},
 {'category_id': 8127,
  'name': 'Брюки',
  'query': 'cat=8127',
  'shard': 'pants',
  'url': 'https://catalog.wb.ru/catalog/pants/v2/catalog?ab_testing=false&appType=1&curr=rub&dest=-446116&sort=popular&spp=30&uclusters=0&cat=8127'},
 {'category_id': 63010,
  'name': 'Верхняя одежда',
  'query': 'cat=63010',
  'shard': 'outwear1',
  'url': 'https://catalog.wb.ru/catalog/outwear1/v2/catalog?ab_testing=false&appType=1&curr=rub&dest=-446116&sort=popular&spp=30&uclusters=0

In [9]:
card_url, success_flag = find_card_url(
    item_id=183884175,
    max_retries=40,
    sleep_time=0.1,
    verbose=True,
    start_basket_number=10,
    max_basket_number=30
)
print(card_url)
if success_flag:
    card = json.loads(requests.get(card_url).content)
    pprint(card)

0: Not https://basket-10.wbbasket.ru/vol1838/part183884/183884175/info/ru/card.json
1: Not https://basket-11.wbbasket.ru/vol1838/part183884/183884175/info/ru/card.json
https://basket-12.wbbasket.ru/vol1838/part183884/183884175/info/ru/card.json
{'certificate': {'verified': True},
 'colors': [204512739, 183884175],
 'compositions': [{'name': 'Хлопок'},
                  {'name': 'Полиэстер'},
                  {'name': 'Вискоза'}],
 'data': {'chrt_ids': [303046544, 303046543, 303026272, 303046542, 303046541],
          'subject_id': 11,
          'subject_root_id': 1},
 'description': 'ВНИМАНИЕ! Брюки XS-XL. Фото № 2 для выбора нужного (до см) '
                'размера. \n'
                '"Обновите свой гардероб с мужскими брюками от CLSS! Наша '
                'коллекция включает в себя стильные и модные брюки на осень, '
                'подходящие как для повседневных, так и официальных '
                'мероприятий. Выбирайте из черных классических брюк с прямыми '
            

In [10]:
from src.parse_utils import (
    find_card_url,
    get_images_urls_from_card_url
)

In [18]:
item_images_urls = get_images_urls_from_card_url(
    card_url=card_url,
    n_images=None,
    verify=False
)
pprint(item_images_urls)

['https://basket-12.wbbasket.ru/vol1838/part183884/183884175/images/big/1.webp',
 'https://basket-12.wbbasket.ru/vol1838/part183884/183884175/images/big/2.webp',
 'https://basket-12.wbbasket.ru/vol1838/part183884/183884175/images/big/3.webp',
 'https://basket-12.wbbasket.ru/vol1838/part183884/183884175/images/big/4.webp',
 'https://basket-12.wbbasket.ru/vol1838/part183884/183884175/images/big/5.webp',
 'https://basket-12.wbbasket.ru/vol1838/part183884/183884175/images/big/6.webp',
 'https://basket-12.wbbasket.ru/vol1838/part183884/183884175/images/big/7.webp',
 'https://basket-12.wbbasket.ru/vol1838/part183884/183884175/images/big/8.webp',
 'https://basket-12.wbbasket.ru/vol1838/part183884/183884175/images/big/9.webp',
 'https://basket-12.wbbasket.ru/vol1838/part183884/183884175/images/big/10.webp']


In [26]:
items_metadata = {}
for i, item_metadata in tqdm(enumerate(browse_result_items['data']['products'], start=1)):
    if i % 15 == 0:
        time.sleep(10)
    item_id = item_metadata['id']
    card_url, success_flag = find_card_url(
        item_id=item_id,
        max_retries=40,
        sleep_time=0.1,
        verbose=False,
        start_basket_number=10,
        max_basket_number=30
    )
    if success_flag:
        items_metadata[item_id] = {'card_url': card_url}
        item_images_urls = get_images_urls_from_card_url(
            card_url=card_url,
            n_images=None,
            verify=True
        )
        items_metadata[item_id]['item_images_urls'] = item_images_urls
pprint(items_metadata)

json.dump(items_metadata, open('./raw_data/browse_category.json', mode='w', encoding='utf-8'))

0it [00:00, ?it/s]

44it [01:31,  1.23s/it]

In [23]:
json.load(open(pjoin(ROOT_PATH, 'raw_data/browse_category.json'), mode='r', encoding='utf-8'))

{'183884175': {'card_url': 'https://basket-12.wbbasket.ru/vol1838/part183884/183884175/info/ru/card.json',
  'item_images_urls': ['https://basket-12.wbbasket.ru/vol1838/part183884/183884175/images/big/1.webp',
   'https://basket-12.wbbasket.ru/vol1838/part183884/183884175/images/big/2.webp',
   'https://basket-12.wbbasket.ru/vol1838/part183884/183884175/images/big/3.webp',
   'https://basket-12.wbbasket.ru/vol1838/part183884/183884175/images/big/4.webp',
   'https://basket-12.wbbasket.ru/vol1838/part183884/183884175/images/big/5.webp',
   'https://basket-12.wbbasket.ru/vol1838/part183884/183884175/images/big/6.webp',
   'https://basket-12.wbbasket.ru/vol1838/part183884/183884175/images/big/7.webp',
   'https://basket-12.wbbasket.ru/vol1838/part183884/183884175/images/big/8.webp',
   'https://basket-12.wbbasket.ru/vol1838/part183884/183884175/images/big/9.webp',
   'https://basket-12.wbbasket.ru/vol1838/part183884/183884175/images/big/10.webp']},
 '250651528': {'card_url': 'https://bask

In [46]:
import typing as t
from src.parse_utils import CatalogType, FlattenCategoriesType
from src.parse_utils import get_browse_categories_with_urls

def get_all_urls(
    catalog: CatalogType,
    save_path: t.Optional[str]=None,
) -> FlattenCategoriesType:
    
    flatten_categories_with_urls = get_browse_categories_with_urls(catalog=catalog)
    for flatten_category in tqdm(flatten_categories_with_urls, desc='flatten_categories_with_urls'):
        url = flatten_category.get('url')
        if url is None:
            continue 
        try:
            category_products = json.loads(requests.get(url).content)
        except Exception as e:
            category_products = []
            continue
        
        items_metadata = {}
        for i, item_metadata in tqdm(enumerate(category_products['data']['products'], start=1)):
            if i % 20 == 0:
                time.sleep(4)
            item_id = item_metadata['id']
            card_url, success_flag = find_card_url(
                item_id=item_id,
                max_retries=40,
                sleep_time=0.1,
                verbose=False,
                start_basket_number=10,
                max_basket_number=30
            )
            if success_flag:
                items_metadata[item_id] = {'card_url': card_url}
                item_images_urls = get_images_urls_from_card_url(
                    card_url=card_url,
                    n_images=None,
                    verify=True
                )
                items_metadata[item_id]['item_images_urls'] = item_images_urls
        flatten_category['products'] = items_metadata
        if save_path is not None:
            json.dump(flatten_categories_with_urls, open(save_path, mode='w', encoding='utf-8'))
    return flatten_categories_with_urls



In [31]:
flatten_categories_with_urls = get_browse_categories_with_urls(catalog=catalog)
for flatten_category in tqdm(flatten_categories_with_urls[:3], desc='flatten_categories_with_urls'):
    url = flatten_category.get('url')
    if url is None:
        continue 
    try:
        category_products = json.loads(requests.get(url).content)
    except Exception as e:
        category_products = []
        continue
    
    items_metadata = {}
    for i, item_metadata in tqdm(enumerate(category_products['data']['products'][:5], start=1)):
        if i % 20 == 0:
            time.sleep(4)
        item_id = item_metadata['id']
        card_url, success_flag = find_card_url(
            item_id=item_id,
            max_retries=40,
            sleep_time=0.1,
            verbose=False,
            start_basket_number=10,
            max_basket_number=30
        )
        if success_flag:
            items_metadata[item_id] = {'card_url': card_url}
            item_images_urls = get_images_urls_from_card_url(
                card_url=card_url,
                n_images=None,
                verify=True
            )
            items_metadata[item_id]['item_images_urls'] = item_images_urls
    flatten_category['products'] = items_metadata

flatten_categories_with_urls:  67%|██████▋   | 2/3 [00:00<00:00, 14.64it/s]


TypeError: string indices must be integers, not 'str'

In [47]:
all_urls_dict = get_all_urls(
    catalog=catalog,
    save_path=pjoin(ROOT_PATH, 'raw_data/all_urls.json')
)
print(f"#all_urls_dict: {len(all_urls_dict):,}")
pprint(all_urls_dict)

flatten_categories_with_urls:   1%|          | 17/2398 [38:25<104:28:46, 157.97s/it]

### bottom